# GEORREFERENCIAMENTO OVP-DH
## Jupyter Notebook compartilhado

In [51]:
import numpy as np
import pandas as pd
import folium
from folium import plugins

### Abrindo dataset

In [52]:
data = pd.read_csv('../../data/amostra_data_geo_bo2016.csv').drop(columns=['Unnamed: 0'])

data.head()

,NUM_BO,DELEGACIA,RUBRICA,MES,LATITUDE,LONGITUDE
0,13,85º D.P. JARDIM MIRNA,Roubo,1,-23.770856,-46.675564
1,185,06º D.P. OSASCO,Furto,1,-23.544059,-46.783202
2,63771,DELEGACIA ELETRONICA,Furto,1,-23.477720,-46.686221
3,117980,DELEGACIA ELETRONICA,Roubo,1,-23.662874,-46.729696
4,55,01º D.P. DIADEMA,Lesão envolvendo veículos,1,-23.712553,-46.606715


### Extraindo amostra do dataset

In [53]:
# o mapa de exemplo nao rodou com o dataset completo

data = data.sample(frac=0.2, random_state=42)

## Construindo o mapa 

### Examinando conteudos de 'RUBRICA'

In [54]:
print('Numero de rubricas: ', data['RUBRICA'].nunique(), '\n')
print('Descricao das rubricas: \n', data['RUBRICA'].unique())

Numero de rubricas:  6 

Descricao das rubricas: 
 ['Roubo' 'Furto' 'Lesão corporal' 'Lesão envolvendo veículos'
 'Crimes envolvendo drogas' 'Homicídio doloso']


In [65]:
# Pegando a média da latitudes e longitudes das ocorrências
latmean = data['LATITUDE'].mean()
lonmean = data['LONGITUDE'].mean()

# Criando o mapa
mapa_popup = folium.Map(location=[latmean,lonmean], 
                         zoom_start=9, 
                         tiles=None, 
                         control_scale=True)

### Bonus: adicionando seletor de texturas

In [56]:
# Determinando as texturas do mapa
texturas = [('openstreetmap', 'textura: tradicional'),
            ('stamenterrain', 'textura: terreno'),
            ('stamenwatercolor','textura: desenho')]

for tile, name in texturas:
    folium.TileLayer(tiles=tile, name=name).add_to(mapa_popup)

### Atribuindo cores diferentes para cada crime

In [57]:
data.RUBRICA.unique()

array(['Roubo', 'Furto', 'Lesão corporal', 'Lesão envolvendo veículos',
       'Crimes envolvendo drogas', 'Homicídio doloso'], dtype=object)

In [58]:
# criando dicionario para receferencia de cores
dic_nl = {'Roubo': 'darkgreen', 
          'Furto': 'lightgreen', 
          'Lesão envolvendo veículos': 'red', 
          'Lesão corporal': 'grey',
          'Homicídio doloso': 'black', 
          'Crime sexual': 'blue',
          'Crimes envolvendo drogas': 'purple',
          'Homicídio envolvendo veículos': 'deepred',}

In [59]:
### Plotando sub-grupos em camadas
all_subgroups = folium.FeatureGroup(name='Tudo')

mapa_popup.add_child(all_subgroups);

In [60]:
# revisando os nomes das colunas para o popup
data.columns

Index(['NUM_BO', 'DELEGACIA', 'RUBRICA', 'MES', 'LATITUDE', 'LONGITUDE'], dtype='object')

In [61]:
html_temp = """
    <h5>CRIME: </h5>
    {}
    <h5>DELEGACIA: </h5>
    {}
    <h5>N B.O.: </h5>
    {}
    """

def grupo_mapa(CRIME):
    group = plugins.FeatureGroupSubGroup(all_subgroups, CRIME)
    mapa_popup.add_child(group)
    
    for i in data[data['RUBRICA'] == CRIME].itertuples():
        crime = i.RUBRICA
        if type(crime) != str:
            crime = "Indefinida"
        delegacia = i.DELEGACIA
        if type(delegacia) != str:
            delegacia = "Indefinida"
        BO = i.NUM_BO
        if type(BO) != str:
            BO = "Indefinido"
        html = html_temp.format(crime, delegacia, BO)
        popup = folium.Popup(html, max_width=280, min_width=250)
        folium.Marker(location=[i.LATITUDE, i.LONGITUDE], 
                      popup=popup, 
                      icon=folium.Icon(color=dic_nl[CRIME], 
                                       icon='bolt', 
                                       prefix='fa')).add_to(group)

In [62]:
#iterando a funcao sobre o set
for i in dic_nl.keys():
    grupo_mapa(i)

/home/nacholez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: color argument of Icon should be one of: {'darkgreen', 'lightblue', 'lightgray', 'lightgreen', 'orange', 'darkred', 'green', 'black', 'pink', 'beige', 'purple', 'darkblue', 'white', 'darkpurple', 'lightred', 'blue', 'cadetblue', 'red', 'gray'}.


### Adicionando menu

In [63]:
folium.LayerControl(collapsed=False, position='bottomright').add_to(mapa_popup);

### Visualizando o mapa

In [64]:
mapa_popup

### Salvando o mapa

In [40]:
mapa_popup.save('../examples/mapa_popup.html')